In [1]:

from mlflow.tracking import MlflowClient

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///prediction.db"
client= MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
client.search_experiments()

[<Experiment: artifact_location='./mlruns/2', creation_time=1666626280346, experiment_id='2', last_update_time=1666626280346, lifecycle_stage='active', name='my-cool-exp', tags={}>,
 <Experiment: artifact_location='./mlruns/1', creation_time=1664455787663, experiment_id='1', last_update_time=1664455787663, lifecycle_stage='active', name='nycity-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/0', creation_time=None, experiment_id='0', last_update_time=None, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
# to create new experiment like we did in the predict_duration.ipynb by using mlflow.set_experiment
client.create_experiment(name="my-cool-exp")

'2'

In [25]:
from mlflow.entities import ViewType
runs = client.search_runs(experiment_ids='1',
    filter_string="",#"metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by=["metrics.rmse ASC"]
    )

In [26]:
runs

[<Run: data=<RunData: metrics={'rmse': 6.320476772946448}, params={'learning_rate': '0.20472',
  'max_depth': '17',
  'min_child_weight': '1.240261172',
  'objective': 'reg:linear',
  'reg_alpha': '0.285678967',
  'reg_lambda': '0.004264404814',
  'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "230f048a415c40d59b562f92cc8efb0d", '
                              '"artifact_path": "model_mlflow", '
                              '"utc_time_created": "2022-10-16 '
                              '12:10:25.259643", "flavors": {"python_function": '
                              '{"loader_module": "mlflow.xgboost", '
                              '"python_version": "3.9.13", "data": "model.xgb", '
                              '"env": "conda.yaml"}, "xgboost": {"xgb_version": '
                              '"1.6.2", "data": "model.xgb", "model_class": '
                              '"xgboost.core.Booster", "code": null}}, '
                              '"model_uuid": '
        

In [27]:
for run in runs:
    print(f"run id:{run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id:230f048a415c40d59b562f92cc8efb0d, rmse: 6.3205
run id:037de7e3b5fb4aa384c6360bee801fc5, rmse: 6.4034
run id:36fdef8ba83c41b493de760818517c46, rmse: 6.4165
run id:579b8c0f24a145cbab9cf304e6c49ab8, rmse: 6.8632
run id:7de858ea07d144acae4ac9af02b5c112, rmse: 6.8839


In [21]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [29]:
run_id ="7de858ea07d144acae4ac9af02b5c112" 
model_uri =f"runs:/{run_id}/model"
# registed a new version of a model 
mlflow.register_model(model_uri=model_uri, name = "nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/10/24 16:14:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1666628061547, current_stage='None', description=None, last_updated_timestamp=1666628061547, name='nyc-taxi-regressor', run_id='7de858ea07d144acae4ac9af02b5c112', run_link=None, source='./mlruns/1/7de858ea07d144acae4ac9af02b5c112/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [31]:
client.search_registered_models()

[<RegisteredModel: creation_timestamp=1666625407463, description='This is NYC taxi predictor for the trip duration!', last_updated_timestamp=1666628061547, latest_versions=[<ModelVersion: creation_timestamp=1666625548429, current_stage='Staging', description='', last_updated_timestamp=1666625892199, name='nyc-taxi-regressor', run_id='579b8c0f24a145cbab9cf304e6c49ab8', run_link='', source='./mlruns/1/579b8c0f24a145cbab9cf304e6c49ab8/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>,
  <ModelVersion: creation_timestamp=1666628061547, current_stage='None', description=None, last_updated_timestamp=1666628061547, name='nyc-taxi-regressor', run_id='7de858ea07d144acae4ac9af02b5c112', run_link=None, source='./mlruns/1/7de858ea07d144acae4ac9af02b5c112/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>]

In [32]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


In [34]:
model_version = 3
stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version = model_version,
    stage = stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1666628061547, current_stage='Staging', description=None, last_updated_timestamp=1666645364652, name='nyc-taxi-regressor', run_id='7de858ea07d144acae4ac9af02b5c112', run_link=None, source='./mlruns/1/7de858ea07d144acae4ac9af02b5c112/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [37]:
from datetime import datetime
date = datetime.today().date()
client.update_model_version(
    name = model_name,
    version = 3,
    description=f"This model version {model_version} was transitioned to {stage} on {date}"
)

<ModelVersion: creation_timestamp=1666628061547, current_stage='Staging', description='This model version 3 was transitioned to Staging on 2022-10-24', last_updated_timestamp=1666645457752, name='nyc-taxi-regressor', run_id='7de858ea07d144acae4ac9af02b5c112', run_link=None, source='./mlruns/1/7de858ea07d144acae4ac9af02b5c112/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>